<a href="https://colab.research.google.com/github/WesselBoi/Suicidal-Content-detection/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -q gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.1 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2;

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from time import time
from collections import Counter

In [ ]:
SEED = 4222
EPOCHS = 5

In [ ]:
# Change to your own directory
try:
    os.chdir("/content/drive/MyDrive/SuicideModel")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

Directory changed


In [ ]:
# Load dataset and reset index
suicide_detection_df = pd.read_csv('Data/suicide_detection_final_cleaned.csv', header=0)
suicide_detection_df.reset_index(drop=True, inplace=True)
suicide_detection_df.replace({"class": {"suicide": 1, "non-suicide": 0}}, inplace=True)
suicide_detection_df.drop(columns=['text'], inplace=True)
suicide_detection_df = suicide_detection_df.rename(columns={"cleaned_text": "text"})
suicide_detection_df

/tmp/ipython-input-3132507467.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  suicide_detection_df.replace({"class": {"suicide": 1, "non-suicide": 0}}, inplace=True)


,class,text
0,1,sex wife threaten suicide recently leave wife ...
1,0,weird not affect compliment come know girl fee...
2,0,finally hear bad year swear fuck god annoying
3,1,need help help cry hard
4,1,end tonight not anymore quit
...,...,...
174847,0,today go sled friend not like pretty big miles...
174848,0,not like rock not go
174849,0,tell friend not lonely deprive buy little nigh...
174850,0,pee probably taste like salty tea drink pee co...


In [ ]:
# Split dataset into train, validation and test sets
train_text, test_text, train_labels, test_labels = train_test_split(suicide_detection_df['text'], suicide_detection_df['class'],
                                                                    random_state=SEED,
                                                                    test_size=0.2,
                                                                    stratify=suicide_detection_df['class'])


#word2vec


Building a vocab


In [ ]:
# define vocab
vocab = Counter()
# tokenise each sentence
tokens_list = [(s.split()) for s in train_text]
# add each sentence to vocab
for i in tokens_list:
  vocab.update(i)
# removing words with a low occurance
min_occurance = 2
tokens = [k for k,c in vocab.items() if c >= min_occurance]
print(len(tokens))

18887


In [ ]:
# save list to file
def save_list(lines, filename):
	# convert lines to a single blob of text
	data = '\n'.join(lines)
	# open file
	file = open(filename, 'w')
	# write text
	file.write(data)
	# close file
	file.close()

# save tokens to a vocabulary file
save_list(vocab, 'Data/vocab.txt')

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load the vocabulary
vocab_filename = 'Data/vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)

Removing out-of-vocab words


In [ ]:
# clean each line
def clean_line(line, vocab):
  tokens = line.split()
  # filter out tokens not in vocab
  tokens_clean = [w for w in tokens if w in vocab]
  return [tokens_clean]

# clean entire dataset
def process_lines(data, vocab):
  lines = list()
  for i in data:
    line = clean_line(i, vocab)
    # add lines to list
    lines += line
  return lines

In [ ]:
train_text = train_text.fillna("").astype(str)
test_text = test_text.fillna("").astype(str)

In [ ]:
train_clean = process_lines(train_text, vocab)
test_clean = process_lines(test_text, vocab)

Training the model

In [ ]:

# set up the parameters of the model
# model = Word2Vec(size=300, window=10, min_count=1, iter=EPOCHS, seed=SEED)
model = Word2Vec(vector_size=300, window=10, min_count=1, epochs=EPOCHS, seed=SEED)

# it builds the vocabulary from a sequence of sentences and thus initialized the model.
t = time()
model.build_vocab(train_clean, progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

# training the model
t = time()
model.train(train_clean, total_examples=model.corpus_count, epochs=EPOCHS, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


Time to build vocab: 0.02 mins
Time to train the model: 0.76 mins


In [ ]:
# save model in ASCII (word2vec) format
filename = 'Data/embedding_word2vec.txt'
model.wv.save_word2vec_format(filename, binary=False)

In [ ]:
model.wv.most_similar('suicide')

[('involuntarily', 0.5647858381271362),
 ('wrongdoing', 0.547781765460968),
 ('firsthand', 0.5384840369224548),
 ('awry', 0.5332929491996765),
 ('clamp', 0.5307917594909668),
 ('arson', 0.5058515667915344),
 ('suicidal', 0.5034372806549072),
 ('unsuccessful', 0.4937410354614258),
 ('institutionalize', 0.4781273901462555),
 ('git', 0.47575780749320984)]